# 使用DirectML进行GPU模型训练
> 我的RX480又可以跑模型了

- toc: false
- badges: true
- comments: true
- hide: true
- categories: [ai]
- image: images/posts/2020-02-25-using-plaidml-keras-to-ai/PlaidML.png

## 关于DirectML
[DirectML](https://github.com/microsoft/DirectML)是Intel的一个AI开发工具，目前支持Keras, ONNX,nGraph。  
现在大火的Tensorflow和PyTorch只支持Nvidia的CUDA进行GPU加速计算。  
而PlaidML可使用OpenCL进行加速。虽然AMD有自己的加速运算平台[ROCm](https://rocm.github.io/)，但目前不支持windows系统，而且OpenCL在速度上貌似还比不上CUDA，对A卡Windows用户（就是我）来说但总比没有的好。

本文使用的机器主要配置如下：
- E3 1230 v2
- RX 480
- DDR3 1333 4G x2

下面从安装到跑模型，来试试PlaidML的效果如何。

## 安装PlaidML
使用pip安装PlaidML: 
> pip instal plaidml-keras  
> plaidml-setup  

根据提示设置PlaidML。

## 使用PlaidML训练Fashion-MNIST分类器
首先用tensorflow中的keras跑一下，看看要跑多久。

In [2]:
# collapse_show
# 使用tensorflow.keras(cpu)训练
import tensorflow as tf
from time import time

data = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = data.load_data()

x_train = x_train.astype('float32').reshape(-1, 28, 28, 1) / 255.
x_test = x_test.astype('float32').reshape(-1, 28, 28, 1) / 255.
# print(x_train.shape)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28, 28, 1)
    ),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(
        filters=32, kernel_size=2, padding='same', activation='relu'
    ),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation='softmax')])
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy'])

train_start = time()
model.fit(x_train, y_train, batch_size=64, epochs=10)
train_end = time()
_, accuracy = model.evaluate(x_test, y_test)
print('training time cost: {0:.1f} s, accuracy: {1:.4f}'.format(train_end-train_start, accuracy))

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 46s 769us/sample - loss: 0.5855 - accuracy: 0.7840
Epoch 2/10
60000/60000 [==============================] - 46s 758us/sample - loss: 0.4013 - accuracy: 0.8541
Epoch 3/10
60000/60000 [==============================] - 46s 762us/sample - loss: 0.3586 - accuracy: 0.8695
Epoch 4/10
60000/60000 [==============================] - 47s 777us/sample - loss: 0.3357 - accuracy: 0.8769
Epoch 5/10
60000/60000 [==============================] - 47s 778us/sample - loss: 0.3172 - accuracy: 0.8826
Epoch 6/10
60000/60000 [==============================] - 44s 736us/sample - loss: 0.3010 - accuracy: 0.8888
Epoch 7/10
60000/60000 [==============================] - 44s 735us/sample - loss: 0.2867 - accuracy: 0.8940
Epoch 8/10
60000/60000 [==============================] - 44s 736us/sample - loss: 0.2781 - accuracy: 0.8974
Epoch 9/10
60000/60000 [==============================] - 44s 737us/sample - loss: 0.2697 - accuracy: 0.9

现在轮到本文的主角plaidml，展现真正的技术了~~（不是~~
> Note: plaidml和tensorflow都有keras，不同的是使用的后端。因此要将keras的后端切换到plaidml，才能确保plaidml正确运行。  

In [1]:
# collapse_show
# 更改keras后端
import plaidml.keras
plaidml.keras.install_backend()
import os
os.environ['KERAS_BACKEND'] = 'plaidml.keras.backend'

> Tip: 如果不想每次都运行上述代码，可在`%USERPROFILE%\.keras\keras.json`配置文件中的`"backend"`设为 `"plaidml.keras.backend"`.

In [1]:
# collapse_show
# 使用plaidml.keras(gpu) 训练
import keras
from time import time

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('float32').reshape(-1, 28, 28, 1)
x_test = x_test.astype('float32').reshape(-1, 28, 28, 1)

model = keras.Sequential([
    keras.layers.Conv2D(
        filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28, 28, 1)
    ),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(
        filters=32, kernel_size=2, padding='same', activation='relu'
    ),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax')])
model.compile(
    optimizer='adam', 
    loss=keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy'])
train_start = time()
model.fit(x_train, y_train, batch_size=64, epochs=10)
train_end = time()
_, accuracy = model.evaluate(x_test, y_test)
print('training time cost: {0:.1f} s, accuracy: {1}'.format(train_end-train_start, accuracy))

Using plaidml.keras.backend backend.
INFO:plaidml:Opening device "opencl_amd_ellesmere.0"
Epoch 1/10
60000/60000 [==============================] - 22s 374us/step - loss: 1.1562 - acc: 0.7010
Epoch 2/10
60000/60000 [==============================] - 12s 193us/step - loss: 0.5358 - acc: 0.8014
Epoch 3/10
60000/60000 [==============================] - 12s 195us/step - loss: 0.4548 - acc: 0.8316
Epoch 4/10
60000/60000 [==============================] - 12s 197us/step - loss: 0.4207 - acc: 0.8457
Epoch 5/10
60000/60000 [==============================] - 11s 185us/step - loss: 0.3989 - acc: 0.8551
Epoch 6/10
60000/60000 [==============================] - 11s 186us/step - loss: 0.3773 - acc: 0.8625
Epoch 7/10
60000/60000 [==============================] - 11s 185us/step - loss: 0.3664 - acc: 0.8651
Epoch 8/10
60000/60000 [==============================] - 11s 187us/step - loss: 0.3551 - acc: 0.8713
Epoch 9/10
60000/60000 [==============================] - 11s 184us/step - loss: 0.3425 - acc:

通过上面两次训练对比看到，CPU训练需要451秒，而通过PlaidML使用GPU训练则只需124秒，大概缩短了1/3的时间，效果还是很明显的。  
综上，PlaidML适合没有N卡但坚守Windows，以及MacBook Pro的用户。但有条件还是搞一台N卡主机吧，

## 番外：为什么不问问神奇的Colab呢？
不试不知道，试了才知道。上面代码在使用了GPU的Colab下跑，结果输出：

> training time cost: 50.0 s, accuracy: 0.9147  
  

最后知道真相的我眼泪流下来，手上PlaidML突然就不香了。
